In [201]:
#ref code from https://www.youtube.com/watch?v=YklplKmCoUw
#importing the Libraries
import numpy as np 
import pandas as pd 
import nltk
import re
from nltk.corpus import stopwords 
import string

In [202]:
#load the data
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [203]:
#read the CSV file
df=pd.read_csv('/content/drive/MyDrive/Test sets for ML/SMSSpamCollection',sep = '\t', header = None)
df.columns = ['label', 'msg']
df.head(10)

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [204]:
print(df.shape)

(5572, 2)


In [205]:
print(df.columns)

Index(['label', 'msg'], dtype='object')


In [206]:
#Pre-processing the dataset
df.isnull().sum()

label    0
msg      0
dtype: int64

In [207]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english')) 

def clean_text(text): 
  #conv to lower case
    text = text.lower() 
    #remove special characters 
    text= re.sub(r'[^0-9a-zA-Z]', ' ', text) 
    #remove spaces
    text= re.sub(r'/s+', ' ', text)
    #remove stopwords
    text = " ".join(word for word in text.split() if word not in STOPWORDS) 
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [208]:
df['clean_text'] = df['msg'].apply(clean_text)
print(df.head) 

<bound method NDFrame.head of      label                                                msg  \
0      ham  Go until jurong point, crazy.. Available only ...   
1      ham                      Ok lar... Joking wif u oni...   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3      ham  U dun say so early hor... U c already then say...   
4      ham  Nah I don't think he goes to usf, he lives aro...   
...    ...                                                ...   
5567  spam  This is the 2nd time we have tried 2 contact u...   
5568   ham               Will ü b going to esplanade fr home?   
5569   ham  Pity, * was in mood for that. So...any other s...   
5570   ham  The guy did some bitching but I acted like i'd...   
5571   ham                         Rofl. Its true to its name   

                                             clean_text  
0     go jurong point crazy available bugis n great ...  
1                               ok lar joking wif u oni  
2     free entr

In [209]:
X=df['clean_text']
y=df['label']

In [210]:
#Use of n-gram model in python ref from various websites
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score

vectorizer = TfidfVectorizer(ngram_range=(2,2)) # (2,2) bigrams
bigrams = vectorizer.fit_transform(df["clean_text"])

In [211]:
bigrams.shape

(5572, 31642)

In [212]:
df["label"].shape

(5572,)

In [213]:
#partially ref from Youtube Video
from sklearn.pipeline import Pipeline

def classify(model, x, y):
#train test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
#model training
  pipeline_model = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf', model)])
  pipeline_model.fit(X_train, y_train)
  print('Accuracy:', model.score(X_test, y_test)*100)
#cv_score = cross_val_score(model, x, y, cv=5)
#print("CV Score:", np.mean(cv_score)*100)
  y_pred = pipeline_model.predict(X_test)
  print(classification_report(y_test, y_pred))

In [ ]:
#Ref from task1 proj
print('Accuracy:', model.score(X_test, y_test)*100)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
pd.DataFrame(
    metrics.confusion_matrix(y_test, y_pred),
    index=[['actual', 'actual'], ['spam', 'ham']],
    columns=[['predicted', 'predicted'], ['spam', 'ham']]
)

In [ ]:
def spam_filter(message):
    if model.predict(vectorizer.transform([clean_text(message)])):
        return 'spam'
    else:
        return 'not spam'

In [ ]:
spam_filter('You have won a guaranteed 32000 award or maybe even £1000 cash to claim ur award call free on 0800 ..... (18+). Its a legitimat efreefone number wat do u think???')

In [ ]:
spam_filter('Dhoni have luck to win some big title.so we will win:)')